In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))

import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile #for audio processing
import os
import pickle
import pandas as pd
from collections import Counter

import librosa   #for audio processing
import librosa.display
# import tensorflow as tf

In [24]:
train_path = "../ALFFA_PUBLIC/ASR/AMHARIC/data/train"
test_path = "../ALFFA_PUBLIC/ASR/AMHARIC/data/test" 

In [25]:
train_audio_folder  = train_path + "/wav/"
train_script_file = train_path + "/trsTrain.txt"
sample_rate = 8000
test_audio_folder  = test_path + "/wav/"
test_script_file = test_path + "/trsTest.txt"

In [26]:
def clean_signal(signal, normalize = False ,trim_db = None, clean_db = None):

    if normalize:
        feats_mean = np.mean(signal, axis=0)
        feats_std = np.std(signal, axis=0)
        signal = (signal - feats_mean) / (feats_std + 1e-14)

    if trim_db:
        signal, index = librosa.effects.trim(signal, top_db=trim_db)

    if clean_db:
        yt = librosa.effects.split(signal, top_db=clean_db)
        clean_signal = []
        for start_i, end_i in yt:
            clean_signal.append( signal[start_i: end_i])

        signal = np.concatenate(np.array(clean_signal),axis=0)

    return signal

In [27]:
# from data_augmentation import Data_Augmentation
from data_loader import DataLoader
data_loader = DataLoader(train_audio_folder, train_script_file, sr=sample_rate)

In [28]:
translation_obj = data_loader.extract_transcription_and_labels()
audio_dict = data_loader.extract_audio(5000)

In [29]:
import helper
helper.write_obj("../data/audio_dict.pkl", audio_dict)
helper.write_obj("../data/translation_dict.pkl", translation_obj)

In [30]:
translation_obj = helper.read_obj("../data/translation_dict.pkl")
audio_obj = helper.read_obj("../data/audio_dict.pkl")
meta_data = data_loader.create_meta_data(translation_obj, audio_obj)

In [31]:
test_data_loader = DataLoader(test_audio_folder, test_script_file, sr=sample_rate)
test_translation_obj = test_data_loader.extract_transcription_and_labels()
test_audio_dict = test_data_loader.extract_audio(20)

helper.write_obj("../data/test_audio_dict.pkl", test_audio_dict)
helper.write_obj("../data/test_translation_dict.pkl", test_translation_obj)

In [35]:
sample_label = list(audio_obj.keys())[490]
translation_obj[sample_label]
ipd.Audio(audio_obj[sample_label][0], rate=sample_rate)

In [36]:
helper.save_csv(meta_data, "../data/meta_data.csv")

File Successfully Saved.!!!


,translation,label,channel,sample_rate,duration
0,የተለያዩ የ ትግራይ አውራጃ ተወላጆች ገንዘባቸው ን አዋጥ ተው የ ልማት ...,tr_10000_tr097082,1,8000,9.088
1,የ ጠመንጃ ተኩስ ተከፈተ ና አራት የኤርትራ ወታደሮች ተገደሉ,tr_10001_tr097083,1,8000,5.632
2,ላነሷቸው ጥያቄዎች የ ሰጡት ን መልስ አቅርበ ነዋል,tr_10002_tr097084,1,8000,6.144
3,እ ብዱ አስፋልቱ ላይ የ ኰለኰ ለ ው ድንጋይ መኪና አላ ሳልፍ አለ,tr_10003_tr097085,1,8000,5.760
4,ጠጁ ን ኰ መኰ መ ኰ መኰ መ ና ሚስቱ ን ሲ ያሰቃ ያት አደረ,tr_10004_tr097086,1,8000,5.376
...,...,...,...,...,...
4995,እነ አቶ ተወልደ ካድሬው የሚ ያግዛቸው መስሏቸው ተስፋ አድርገው እንደ ነ...,tr_4709_tr48010,1,8000,7.936
4996,የ አውሮፓ ፓር ሊያ ሜንት በ ኢትዮጵያ ዲሞክራሲያዊ ሂደት ላይ ያለውን ጥ...,tr_470_tr05070,1,8000,6.272
4997,አንቺ ቆንጆ የምት ያቸው እንዴት ያሉት ን ሴቶች ነው ለስላሳ ሳቅ በ አዳ...,tr_4710_tr48011,1,8000,5.632
4998,በ አይ ኤም ኤፍ የ ዋሽንግተን ስብሰባ የ ኢትዮጵያ ልኡክ ሊ ሄድ ነው,tr_4711_tr48012,1,8000,4.480


In [37]:
sample_audio_obj = audio_obj['tr_10032_tr097114']
sample_audio = sample_audio_obj[0]
rate = sample_audio_obj[-1]